# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), January 18, 2019**

In [1]:
using DataFrames # load package

## Работа с CategoricalArrays

### Constructor

In [2]:
x = categorical(["A", "B", "B", "C"]) # неупорядоченный

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [3]:
y = categorical(["A", "B", "B", "C"], ordered=true) # упорядоченный, по умолчанию порядок сортировки

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [4]:
z = categorical(["A","B","B","C", missing]) # неупорядоченный с пропусками

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "A"    
 "B"    
 "B"    
 "C"    
 missing

In [5]:
c = cut(1:10, 5) # ordered, into equal counts, possible to rename labels and give custom breaks

10-element CategoricalArray{String,1,UInt32}:
 "[1.0, 2.8)" 
 "[1.0, 2.8)" 
 "[2.8, 4.6)" 
 "[2.8, 4.6)" 
 "[4.6, 6.4)" 
 "[4.6, 6.4)" 
 "[6.4, 8.2)" 
 "[6.4, 8.2)" 
 "[8.2, 10.0]"
 "[8.2, 10.0]"

In [6]:
by(DataFrame(x=cut(randn(100000), 10)), :x, d -> DataFrame(n=nrow(d)), sort=true) 
# просто чтобы убедиться, что он работает правильно

10×2 DataFrame
│ Row │ x                       │ n     │
│     │ Categorical…            │ Int64 │
├─────┼─────────────────────────┼───────┤
│ 1   │ [-4.58526, -1.26962)    │ 10000 │
│ 2   │ [-1.26962, -0.833354)   │ 10000 │
│ 3   │ [-0.833354, -0.521999)  │ 10000 │
│ 4   │ [-0.521999, -0.250324)  │ 10000 │
│ 5   │ [-0.250324, 0.00390711) │ 10000 │
│ 6   │ [0.00390711, 0.253528)  │ 10000 │
│ 7   │ [0.253528, 0.526083)    │ 10000 │
│ 8   │ [0.526083, 0.841457)    │ 10000 │
│ 9   │ [0.841457, 1.27925)     │ 10000 │
│ 10  │ [1.27925, 4.27429]      │ 10000 │

In [7]:
v = categorical([1,2,2,3,3]) # содержит целые числа, а не строки

5-element CategoricalArray{Int64,1,UInt32}:
 1
 2
 2
 3
 3

In [8]:
Vector{Union{String, Missing}}(z) # иногда вам нужно преобразовать обратно в стандартный вектор

5-element Array{Union{Missing, String},1}:
 "A"    
 "B"    
 "B"    
 "C"    
 missing

### Управление уровнями

In [9]:
arr = [x,y,z,c,v]

5-element Array{CategoricalArray{T,1,UInt32,V,C,U} where U where C where V where T,1}:
 CategoricalString{UInt32}["A", "B", "B", "C"]                                                                                                                          
 CategoricalString{UInt32}["A", "B", "B", "C"]                                                                                                                          
 Union{Missing, CategoricalString{UInt32}}["A", "B", "B", "C", missing]                                                                                                 
 CategoricalString{UInt32}["[1.0, 2.8)", "[1.0, 2.8)", "[2.8, 4.6)", "[2.8, 4.6)", "[4.6, 6.4)", "[4.6, 6.4)", "[6.4, 8.2)", "[6.4, 8.2)", "[8.2, 10.0]", "[8.2, 10.0]"]
 CategoricalValue{Int64,UInt32}[1, 2, 2, 3, 3]                                                                                                                          

In [10]:
isordered.(arr) # проверяет, упорядочен ли массив категорий

5-element BitArray{1}:
 false
  true
 false
  true
 false

In [11]:
ordered!(x, true), isordered(x) # упорядочит х

(CategoricalString{UInt32}["A", "B", "B", "C"], true)

In [12]:
ordered!(x, false), isordered(x) # и снова разупорядочит

(CategoricalString{UInt32}["A", "B", "B", "C"], false)

In [13]:
levels.(arr) # список уровней

5-element Array{Array{T,1} where T,1}:
 ["A", "B", "C"]                                                        
 ["A", "B", "C"]                                                        
 ["A", "B", "C"]                                                        
 ["[1.0, 2.8)", "[2.8, 4.6)", "[4.6, 6.4)", "[6.4, 8.2)", "[8.2, 10.0]"]
 [1, 2, 3]                                                              

In [14]:
unique.(arr) # пропуски будут включены

5-element Array{Array{T,1} where T,1}:
 ["A", "B", "C"]                                                        
 ["A", "B", "C"]                                                        
 Union{Missing, String}["A", "B", "C", missing]                         
 ["[1.0, 2.8)", "[2.8, 4.6)", "[4.6, 6.4)", "[6.4, 8.2)", "[8.2, 10.0]"]
 [1, 2, 3]                                                              

In [15]:
y[1] < y[2] # проверить как y упорядочен

true

In [16]:
v[1] < v[2] # несопоставимо, v неупорядочено, хотя оно содержит целые числа

ArgumentError: ArgumentError: Unordered CategoricalValue objects cannot be tested for order using <. Use isless instead, or call the ordered! function on the parent array to change this

In [17]:
y[2] < "A" # возможно сравнение с типом, лежащим в основе категориального значения

false

In [18]:
y[2] < "Z" # но он рассматривается как уровень, и поэтому разрешены только допустимые уровни

KeyError: KeyError: key "Z" not found

In [19]:
levels!(y, ["C", "B", "A"]) 
# Вы можете изменить порядок уровней, в основном это полезно для упорядоченных массивов категорий

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [20]:
y[1] < y[2] # обратите внимание, что порядок изменился

false

In [21]:
y[1] < "B" # порядок уровней также учитывается при сравнении с базовым типом

false

In [22]:
levels!(z, ["A", "B"]) # Вы должны указать все уровни, которые присутствуют

ArgumentError: ArgumentError: cannot remove level "C" as it is used at position 4 and allow_missing=false.

In [23]:
levels!(z, ["A", "B"], allow_missing=true) # если базовый массив не позволяет миссинги и принудительно удалить уровни

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "A"    
 "B"    
 "B"    
 missing
 missing

In [24]:
z[1] = "B"
z # теперь z имеет только записи 'B'

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "B"    
 "B"    
 "B"    
 missing
 missing

In [25]:
levels(z) # но он помнит уровни, которые у него были (причина в основном производительность)

2-element Array{String,1}:
 "A"
 "B"

In [26]:
droplevels!(z) # таким образом, мы можем очистить его
levels(z)

1-element Array{String,1}:
 "B"

### Манипуляция данными

In [27]:
x, levels(x)

(CategoricalString{UInt32}["A", "B", "B", "C"], ["A", "B", "C"])

In [28]:
x[2] = "0"
x, levels(x) # добавлен новый уровень в конце (работает только для неупорядоченных)

(CategoricalString{UInt32}["A", "0", "B", "C"], ["A", "B", "C", "0"])

In [29]:
v, levels(v)

(CategoricalValue{Int64,UInt32}[1, 2, 2, 3, 3], [1, 2, 3])

In [30]:
v[1] + v[2] # даже если базовые данные Int, мы не можем работать с ними

MethodError: MethodError: no method matching +(::CategoricalValue{Int64,UInt32}, ::CategoricalValue{Int64,UInt32})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

In [31]:
Vector{Int}(v) # Вы должны либо получить данные путем преобразования (может быть дорого)

5-element Array{Int64,1}:
 1
 2
 2
 3
 3

In [32]:
get(v[1]) + get(v[2]) или получить единственное значение

3

In [33]:
get.(v) # это будет работать для массивов без пропусков

5-element Array{Int64,1}:
 1
 2
 2
 3
 3

In [34]:
get.(z) # но потерпит неудачу при пропущенных значениях

MethodError: MethodError: no method matching get(::Missing)
Closest candidates are:
  get(!Matched::Base.EnvDict, !Matched::AbstractString, !Matched::Any) at env.jl:77
  get(!Matched::Base.TTY, !Matched::Symbol, !Matched::Any) at stream.jl:411
  get(!Matched::REPL.Terminals.TTYTerminal, !Matched::Any, !Matched::Any) at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\REPL\src\Terminals.jl:176
  ...

In [35]:
Vector{Union{String, Missing}}(z) # вы должны сделать преобразование

5-element Array{Union{Missing, String},1}:
 "B"    
 "B"    
 "B"    
 missing
 missing

In [36]:
z[1]*z[2], z.^2
# единственное исключение - это Категориальные массивы на основе String - вы можете работать с ними как с обычными строками

("BB", Union{Missing, String}["BB", "BB", "BB", missing, missing])

In [37]:
recode([1,2,3,4,5,missing], 1=>10) # recode some values in an array; has also in place recode! equivalent

6-element Array{Union{Missing, Int64},1}:
 10       
  2       
  3       
  4       
  5       
   missing

In [38]:
recode([1,2,3,4,5,missing], "a", 1=>10, 2=>20) # здесь мы предоставили значение по умолчанию для не отображенных записей

6-element Array{Any,1}:
 10       
 20       
   "a"    
   "a"    
   "a"    
   missing

In [39]:
recode([1,2,3,4,5,missing], 1=>10, missing=>"missing") # перекодировать пропуски явно

6-element Array{Any,1}:
 10         
  2         
  3         
  4         
  5         
   "missing"

In [40]:
t = categorical([1:5; missing])
t, levels(t)

(Union{Missing, CategoricalValue{Int64,UInt32}}[1, 2, 3, 4, 5, missing], [1, 2, 3, 4, 5])

In [41]:
recode!(t, [1,3]=>2)
t, levels(t) # обратите внимание, что уровни сбрасываются после перекодирования

(Union{Missing, CategoricalValue{Int64,UInt32}}[2, 2, 2, 4, 5, missing], [2, 4, 5])

In [42]:
t = categorical([1,2,3], ordered=true)
levels(recode(t, 2=>0, 1=>-1)) # и если вы вводите новые уровни, они добавляются в конце в порядке появления

3-element Array{Int64,1}:
  3
  0
 -1

In [43]:
t = categorical([1,2,3,4,5], ordered=true) # при использовании по умолчанию становится последним уровнем
levels(recode(t, 300, [1,2]=>100, 3=>200))

3-element Array{Int64,1}:
 100
 200
 300

### Сравнения

In [44]:
x = categorical([1,2,3])
xs = [x, categorical(x), categorical(x, ordered=true), categorical(x, ordered=true)]
levels!(xs[2], [3,2,1])
levels!(xs[4], [2,3,1])
[a == b for a in xs, b in xs] # все равны - сравнение только по содержанию

4×4 Array{Bool,2}:
 true  true  true  true
 true  true  true  true
 true  true  true  true
 true  true  true  true

In [45]:
signature(x::CategoricalArray) = (x, levels(x), isordered(x)) # это на самом деле полная подпись CategoricalArray
# разных элементов, обратите внимание, что x [1] и x [2] неупорядочены, но имеют другой порядок уровней
[signature(a) == signature(b) for a in xs, b in xs]

4×4 Array{Bool,2}:
  true  false  false  false
 false   true  false  false
 false  false   true  false
 false  false  false   true

In [46]:
x[1] < x[2] # Вы не можете сравнивать элементы неупорядоченного массива

ArgumentError: ArgumentError: Unordered CategoricalValue objects cannot be tested for order using <. Use isless instead, or call the ordered! function on the parent array to change this

In [47]:
t[1] < t[2] # но вы можете сделать это для упорядоченного

true

In [48]:
isless(x[1], x[2]) # isless работает в том же массиве категорий, даже если он не упорядочен

true

In [49]:
y = deepcopy(x) # но не по категориальным массивам
isless(x[1], y[2])

ArgumentError: ArgumentError: CategoricalValue objects with different pools cannot be tested for order

In [50]:
isless(get(x[1]), get(y[2])) # вы можете использовать get для сравнения содержимого CategoryoricalArray

true

In [51]:
x[1] == y[2] # Тесты на равенство работают нормально во всех CategoricalArrays

false

### Категориальные столбцы в DataFrame

In [52]:
df = DataFrame(x = 1:3, y = 'a':'c', z = ["a","b","c"])

3×3 DataFrame
│ Row │ x     │ y    │ z      │
│     │ Int64 │ Char │ String │
├─────┼───────┼──────┼────────┤
│ 1   │ 1     │ 'a'  │ a      │
│ 2   │ 2     │ 'b'  │ b      │
│ 3   │ 3     │ 'c'  │ c      │

In [53]:
categorical!(df) # преобразует все столбцы eltype (AbstractString) в категориальные

3×3 DataFrame
│ Row │ x     │ y    │ z            │
│     │ Int64 │ Char │ Categorical… │
├─────┼───────┼──────┼──────────────┤
│ 1   │ 1     │ 'a'  │ a            │
│ 2   │ 2     │ 'b'  │ b            │
│ 3   │ 3     │ 'c'  │ c            │

In [54]:
describe(df)

3×8 DataFrame. Omitted printing of 1 columns
│ Row │ variable │ mean   │ min │ median │ max │ nunique │ nmissing │
│     │ Symbol   │ Union… │ Any │ Union… │ Any │ Union…  │ Nothing  │
├─────┼──────────┼────────┼─────┼────────┼─────┼─────────┼──────────┤
│ 1   │ x        │ 2.0    │ 1   │ 2.0    │ 3   │         │          │
│ 2   │ y        │        │ 'a' │        │ 'c' │ 3       │          │
│ 3   │ z        │        │ a   │        │ c   │ 3       │          │

In [55]:
categorical!(df, :x) # вручную преобразовать в категориальный столбец :x

3×3 DataFrame
│ Row │ x            │ y    │ z            │
│     │ Categorical… │ Char │ Categorical… │
├─────┼──────────────┼──────┼──────────────┤
│ 1   │ 1            │ 'a'  │ a            │
│ 2   │ 2            │ 'b'  │ b            │
│ 3   │ 3            │ 'c'  │ c            │

In [56]:
describe(df)

3×8 DataFrame. Omitted printing of 1 columns
│ Row │ variable │ mean    │ min │ median  │ max │ nunique │ nmissing │
│     │ Symbol   │ Nothing │ Any │ Nothing │ Any │ Int64   │ Nothing  │
├─────┼──────────┼─────────┼─────┼─────────┼─────┼─────────┼──────────┤
│ 1   │ x        │         │ 1   │         │ 3   │ 3       │          │
│ 2   │ y        │         │ 'a' │         │ 'c' │ 3       │          │
│ 3   │ z        │         │ a   │         │ c   │ 3       │          │